In [ ]:
#importar librerias
#peticion al servidor
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='https://www.elsevier.es/en-revista-annals-hepatology-16-numeros-anteriores-anio-2002'
url_root= 'https://www.elsevier.es/en-revista-annals-hepatology-16-numeros-anteriores-anio-2002'
r = requests.get(url_inicial)

In [ ]:
#Se realiza la busquea de la primeras etiquetas
#para obtener lo href de cada uno de lo volumenes o archivos de la revista.
soup = BeautifulSoup(r.text, 'html.parser')
volumen = soup.find('div', class_='sumarios decada-2002')
vol=volumen.findAll('div',class_='portada cell')
vo = [x.find('a').get('href')for x in vol]
vo=[urljoin(url_root,i) for i in vo]
vo

In [ ]:
#Se realiza un lista en la que la variable vol contiene los principales href
#con estos href obtenidos, se implemeta un ciclo de esto para con ello obtener
#los siguentes href de los arcticulos que se encuentran en cada uno de los volumenes o archivos
#para esto se realiza la misma busqueda que al principio.
vol2 =[]
for i in vo: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item-preview')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1=[urljoin(url_root,i) for i in vol1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
#En esta parte la funcion es de obtener links
def get_url_items(sopa, url): 
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item-preview')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1=[urljoin(url_root,i) for i in vol1]
    return vol2

In [ ]:
#Ahora se va acumulando cada uno de lo links e ir iterando en cada una
#de las páginas, para traer cada unos de los links que se van a ir scrapeando.
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
#Se hace un listado de los links de lo articulos que se obtuvieron.
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
#Se estaran tomado uno a uno los links para Scrapearlos.
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
#Mediante esta función resivi cada uno de los links, va ir aplicando caca uno de 
#los metodos, que nos van a obtener lo que estamos requiriendo.
def scraper_book(url):
    content_book={}#diccionario que no va acumulando la imformacion que vamos poteniddo para el tada frame
    r=requests.get(url)
    tr='Annals of Hepatology'
    a='III. Medicina y Ciencias de la Salud'
    te='Hepatología'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', id='resumen_es').get_text(strip=True)
        content_book['Resumen']=des.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', id='resumen_en').get_text(strip=True)
        content_book['Abstract']=de.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=url
        content_book['Link Articulo']=urlA#join(url_root,urlA)
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='item-anchors desktop').find('a').get('href')#.find('div', class_='opcion imprimir')
        content_book['Link PDF']=urljoin(url_root,ancla_link)#.find_next_sibling('a').get('href')
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
#El ciclo hace un scrapeo de la cada uno de los articulos.
list_scraper=list_scraper[0:33]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
#En listado de datos_book que se obtuvo, se guarada en una variable
#y dicha variable que conteien la imformacion se convierte en un DataFrame.
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
#En el caso de esta revista solo de deseaba obtener los articulos,por lo cual
#en algunas ocaciones se obtenia cosa diferntes, por lo cual esta funcion lo que hace es 
#eliminar mediante el Tiitulo Articulo lo que no se desea.
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","RESEÑAS","indice","In memoriam","Resumen","RESUMENES","Horizontes","ACERCA DE LOS AUTORES","In Memoriam","Complate journal","Preliminary","Preliminaries","Complete Journal","NOVEDADES EDITORIALES","Autores","Fondo Greimas","Resúmenes","Índice","-","Anticipos y deudas","EDITORIAL","Reseñas","Acerca de los autores","Acerca de Autores","Acerca de los Autores","ÍNDICE","RESÚMENES","Preface",None,"In Reply","Yellow Fever","News","Hepatology highlights","An Acknowledgement","Hepatology Highlights","Acute Liver Failure","From the editor","An Acknowledgment","The rolling stone","Letter to the Editor","Editor’s Report","A warm welcome"])]
df

In [ ]:
#La imformacion conseguda se pasa a un CSV.
df.to_csv('Revista002.20.csv', index=False)

Final del Código